In [3]:
import pandas as pd

In [5]:
content = pd.read_csv('Content.csv')
reaction = pd.read_csv('Reactions.csv')
reaction_type = pd.read_csv('ReactionTypes.csv')

In [7]:
df1 = content.copy()
df2 = reaction.copy()
df3 = reaction_type.copy()

In [9]:
df1 = df1.drop(columns=['User ID', 'URL', 'Unnamed: 0'])
df1 = df1.rename(columns={'Type': 'Content Type'})

In [11]:
df2.head()

,Unnamed: 0,Content ID,User ID,Type,Datetime
0,0,97522e57-d9ab-4bd6-97bf-c24d952602d2,NaN,NaN,2021-04-22 15:17:15
1,1,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:50
2,2,97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike,2021-06-17 12:22:51
3,3,97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared,2021-04-18 05:13:58
4,4,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:01


In [13]:
df2 = df2.drop(columns=['Datetime', 'Unnamed: 0'])

In [15]:
df2 = df2.rename(columns={'Type': 'Reaction Type'})

In [17]:
df2.head()

,Content ID,User ID,Reaction Type
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,NaN,NaN
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust


In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25553 entries, 0 to 25552
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Content ID     25553 non-null  object
 1   User ID        22534 non-null  object
 2   Reaction Type  24573 non-null  object
dtypes: object(3)
memory usage: 599.0+ KB


In [21]:
pd.options.display.max_rows = None

In [23]:
df1['Category'] = df1['Category'].str.replace('"', '')

In [25]:
df1_final = df1.copy()

In [27]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25553 entries, 0 to 25552
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Content ID     25553 non-null  object
 1   User ID        22534 non-null  object
 2   Reaction Type  24573 non-null  object
dtypes: object(3)
memory usage: 599.0+ KB


In [29]:
df2 = df2.dropna(subset=['Reaction Type'])
df2['User ID'] = df2['User ID'].fillna('Unknown')

In [31]:
df2_final = df2.copy()

In [33]:
df3 = df3.rename(columns={'Type': 'Reaction Type'})
df3 = df3.rename(columns={'Score': 'Reaction Popularity'})

In [35]:
df3_final = df3.copy()

In [37]:
df2_df1 = pd.merge(df2_final, df1_final, on='Content ID', how='left')
df_semifinal = pd.merge(df2_df1, df3_final, on='Reaction Type', how='left')

In [39]:
df_concat = pd.concat([df2_df1, df_semifinal], axis=1)
df_final = df_concat[['Content ID', 'Content Type', 'Category', 'Reaction Type', 'Reaction Popularity']]

In [41]:
df_final = df_final.loc[:, ~df_final.columns.duplicated()]
df_final.head()

,Content ID,Content Type,Category,Reaction Type,Reaction Popularity
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,Studying,disgust,0
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,Studying,dislike,10
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,Studying,scared,15
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,Studying,disgust,0
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,photo,Studying,interested,30


In [43]:
df_final['Category'] = df_final['Category'].str.lower().str.strip()
top_categories = df_final.groupby('Category')['Reaction Popularity'].sum().reset_index()
top_categories = top_categories.sort_values(by='Reaction Popularity', ascending=False).head(5)

# now let's check, everything else is done

print(top_categories)

          Category  Reaction Popularity
0          animals                74965
9          science                71168
7   healthy eating                69339
12      technology                68738
6             food                66676


In [45]:
df_final.to_csv('df_final_preprocessed.csv', index=False)
